In [5]:
'''!pip install Groq -q
!pip install groq -q'''

In [37]:
TEST_PATH = '../input/Raw Text/FAQ/faq.json'
LEGIS_PATH = '../input/Raw Text/Legislation/output_legislation_2.json'
NORMS_PATH = '../input/Raw Text/Norms/output_prof3_v10_5_final.json'

In [38]:
import json

test_set = json.load(open(TEST_PATH, 'r'))
legis_set = json.load(open(LEGIS_PATH, 'r'))
norms_set = json.load(open(NORMS_PATH, 'r'))

In [34]:
type(legis_set)

list

In [3]:
len(norms_set), len(legis_set), len(test_set)


(4650, 10, 33)

In [4]:
from groq import Groq

client = Groq(api_key='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')

def groq_chat(content):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": content,
            }
        ],
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content

In [5]:
# Inspirado no código do Leandro Carísio

perg_resposta_esperada = []

for i in range(len(test_set)):
  pr = test_set[i]

  pergunta = pr['question']
  resposta = pr['answer']
  #contexto = [item['text'] for item in pr['context']]
  #contexto = ', '.join(contexto)
 
  perg_resposta_esperada.append({"pergunta": pergunta, "resposta": resposta})

  if i < 5:
    print(f"{i}: {pergunta}\n{resposta}")
    print('----------------------------------------------------------------------')

0: Em quais hipóteses a Procuradoria Geral pode exercer a representação judicial e extrajudicial de servidores da UNICAMP?
A Procuradoria Geral da UNICAMP integra a Advocacia Pública e está vinculada à Procuradoria Geral do Estado, para fins de atuação uniforme e coordenada, nos termos do artigo 101 da Constituição Estadual. Além disso, é o órgão de representação jurídica da Universidade e de assessoramento jurídico da Reitoria, consoante prevê o artigo 95 do Regimento Geral da UNICAMP. Desta forma, conforme previsão regimental, a Procuradoria Geral tem a função institucional de defender os interesses da universidade, enquanto autarquia estadual, estritamente vinculada ao atendimento do interesse público. A representação judicial ou extrajudicial de servidor com vínculo funcional permanente poderá ocorrer em caráter excepcional, nas hipóteses em que o ato impugnado decorrer do exercício de suas atribuições constitucionais, legais ou regulamentares, quando o ato praticado tiver seguido 

In [6]:
QAC_generator_prompt = """You are a program that generates questions, answers to those questions, and context you used to answer those questions.
You will receive many text documents in .json format with "title" and "text". For each document, you will generate a question, an answer and the context
for this answer, that is, where in the text you can check the veracity of that answer.

Your response will be in .json format, translated to Brazillian Portuguese,


title: {title}
text: {text}
"""

In [7]:
legis_set[3]['titulo']

'DECRETO Nº 29.598, DE 02 DE FEVEREIRO DE 1989'

In [ ]:
# %%time
# documents = []
# all_titles = []

# for item in test_set:
#     if item['title'].lower() not in all_titles:
#         documents.append({
#                 "title": item['title'],
#                 "content": item["text"]
#             }
#         )
#         all_titles.append(item['title'].lower())
#     for link in item["links"]:
#         if link['target'].lower() in context_articles and link['target'].lower() not in all_titles:
#             documents.append({
#                 "title": link['target'],
#                 "content": context_articles[link['target'].lower()]
#             })
#             all_titles.append(link['target'].lower())

In [16]:
dummy_test_faq =[
    {
        "question": "Em quais hipóteses a Procuradoria Geral pode exercer a representação judicial e extrajudicial de servidores da UNICAMP?",
        "answer": "A Procuradoria Geral da UNICAMP integra a Advocacia Pública e está vinculada à Procuradoria Geral do Estado, para fins de atuação uniforme e coordenada, nos termos do artigo 101 da Constituição Estadual. Além disso, é o órgão de representação jurídica da Universidade e de assessoramento jurídico da Reitoria, consoante prevê o artigo 95 do Regimento Geral da UNICAMP. Desta forma, conforme previsão regimental, a Procuradoria Geral tem a função institucional de defender os interesses da universidade, enquanto autarquia estadual, estritamente vinculada ao atendimento do interesse público. A representação judicial ou extrajudicial de servidor com vínculo funcional permanente poderá ocorrer em caráter excepcional, nas hipóteses em que o ato impugnado decorrer do exercício de suas atribuições constitucionais, legais ou regulamentares, quando o ato praticado tiver seguido a orientação jurídica da Procuradoria e desde que não haja conflito de interesses, sempre com autorização expressa do Reitor.",
        "context": [
            {
                "text": 'texto do regimento',
                "title": "REGIMENTO GERAL DA UNIVERSIDADE ESTADUAL DE CAMPINAS",
            },
            {
                "text": "texto da constituição",
                "title": "Constituição Federal",
            }
    ],
    }
]

In [44]:
all_titles = []
documents = []


for item in dummy_test_faq:
    for context in item['context']:
        if context['title'] not in all_titles:
            all_titles.append(context['title'].lower())
            documents.append({
                    "title": context['title'],
                    "content": legis_set[context['title']] if context['title'] in legis_set else norms_set[context['title']]
                }
            )

In [52]:
%%time
from tqdm import tqdm
import spacy
import pickle

nlp = spacy.blank("pt")
nlp.add_pipe("sentencizer")

stride = 2
max_length = 3

def window(documents, stride=2, max_length=3):
  treated_documents = []

  for j, document in enumerate(tqdm(documents)):
    doc_text = document['content']
    doc = nlp(doc_text)
    sentences = [sent.text.strip() for sent in doc.sents]
    for i in range(0, len(sentences), stride):
      segment = ' '.join(sentences[i:i + max_length])
      treated_documents.append({
          "title": document['title'],
          "contents": document['title']+". "+segment,
          "segment": segment
      })
      if i + max_length >= len(sentences):
        break
  return treated_documents


artigos_segmentados = window(documents)

100%|██████████| 2/2 [00:02<00:00,  1.00s/it]

CPU times: total: 1.11 s
Wall time: 2.1 s


In [53]:
%%time
from collections import Counter
import array
import pickle
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

# Definição de uma classe para índice invertido
class IndiceInvertido:

  # Recebe 'tokenizar', uma função tokenizadora
  def __init__(self):
    # Cria um índice invertido vazio
    self.indice = {}
    # Cria um índice de tamanho de documentos vazio
    self.tamanho_doc = {}
    # Guarda o total de documentos adicionados
    self.n_docs = 0
    # Tokenizador
    self.stemmer = PorterStemmer()

  def tokenizar(self, texto):
    # Tokeniza
    tokens = word_tokenize(texto)
    # Converte para minúsculo, filtra stopwords e passa pelo Porter Stemer
    tokens = [self.stemmer.stem(token.lower()) for token in tokens if token.lower() not in stopwords.words('english')]

    return tokens

  def adiciona_doc(self, id_doc, conteudo_doc=None):
    tokens = self.tokenizar(conteudo_doc)

    contador_tokens_do_documento = Counter(tokens)
    for token, n_ocorrencias in contador_tokens_do_documento.items():
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['id_doc'].append(id_doc)
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['n_ocorrencias'].append(n_ocorrencias)

    self.n_docs += 1
    self.tamanho_doc[id_doc] = len(tokens)

class BM25:

  def __init__(self, indiceInvertido=IndiceInvertido(), k1 = 0.9, b = 0.4, bias_adicionar_ao_idf = 0):
    self.indiceInvertido = indiceInvertido
    self.bias_adicionar_ao_idf = bias_adicionar_ao_idf
    self.calcula_tam_medio_doc_no_indice()
    self.k1 = k1
    self.b = b
    self.precalcula_idf()
    self.reinicia_score_dos_indices()

  def reinicia_score_dos_indices(self):
    for token in self.indiceInvertido.indice.keys():
      self.indiceInvertido.indice[token].pop('score', None)

  def calcula_tam_medio_doc_no_indice(self):
    self.avgdl = sum(self.indiceInvertido.tamanho_doc.values()) / self.indiceInvertido.n_docs

  def precalcula_idf(self):
    # Número de documento do corpus está presente no objeto indiceInvertido
    N = self.indiceInvertido.n_docs
    # Varre todos os tokens do índice. Os tokens são as chaves do indiceInvertido.indice
    for token in self.indiceInvertido.indice.keys():
      # O número de documentos que possui o token é calculado pelo tamanho da lista de id_doc:
      n_doc_token = len(self.indiceInvertido.indice[token]['id_doc'])
      # Isso já é o suficiente pra calcular o idf
      idf_token = math.log( ((self.indiceInvertido.n_docs - n_doc_token + 0.5)/(n_doc_token + 0.5)) + self.bias_adicionar_ao_idf )
      # E agora, vamos colocar essa informação no índice
      self.indiceInvertido.indice[token]['idf'] = idf_token

  def calcula_score_para_um_token_e_salva(self, token):
    # O cálculo do BM25 para determinada query é a multiplicação do idf pela frequência do termo no documento * (k1 + 1)
    # Além disso, é dividido pela frequencia do termo no documento + k1 * (1 - b + b * tamanho_doc/avgdl)
    idf = self.indiceInvertido.indice[token]['idf']
    # Juntando tudo, podemos calcular o score pelo BM25
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])
    bm25 = array.array("f", [ idf * freq_token_no_doc * (self.k1 + 1) / (freq_token_no_doc + self.k1 * (1 - self.b + self.b * self.indiceInvertido.tamanho_doc[id_doc] / self.avgdl)) for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o bm25 no índice
    self.indiceInvertido.indice[token]['score'] = bm25

  def tokenizar(self, query):
    return self.indiceInvertido.tokenizar(query)

  def pesquisar(self, query):
    # Tokeniza a query
    tokens = self.tokenizar(query)

    # Se não tem token para ser pesquisado, retorna conjunto vazio
    if (len(tokens) == 0):
      return []

    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    for token in tokens:
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
      # entende-se que a frequência desse token em qualquer documento é 0, já que não pode ser encontrado
      if token not in self.indiceInvertido.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indiceInvertido.indice[token]['id_doc']

      # Se for a primeira vez que esse token é pesquisado, é necessário calcular o score relacionado
      # a ele e salvar. Se já tiver sido feito antes, já podemos buscar o cálculo pronto (que funciona
      # como um cache. Isso é útil no caso de várias pesquisas seguidas)
      if 'score' not in self.indiceInvertido.indice[token].keys():
        self.calcula_score_para_um_token_e_salva(token)
      score_dos_docs_deste_token = self.indiceInvertido.indice[token]['score']

      # Agora já temos calculado o score de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      for id_doc, score_par_doc_token in zip(docs_que_tem_token, score_dos_docs_deste_token):
        docs_retornado_com_score[id_doc] += score_par_doc_token

    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

CPU times: total: 250 ms
Wall time: 1.72 s


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
iidx = IndiceInvertido()
iidx.tokenizar('Qual o maior planeta do sistema solar?')

['qual', 'maior', 'planeta', 'sistema', 'solar', '?']

In [57]:
# Se True, gera o índice invertido e salva em um pickle. Se False, apenas recupera o pickle do drive
GERAR_INDICE_INVERTIDO = True
ARQUIVO_INDICE_INVERTIDO = 'iidx_artigos_segmentados' # A função salva em 2 arquivos, então ela mesma coloca a extensão pickle

# Se True, gera as respostas. Se False, apenas recupera o pickle do drive
AGREGAR_DOCUMENTOS = True
ARQUIVO_AGREGADO = 'respostas_obtidas.pickle'

In [66]:
import pickle

def salvar_indice_em_arquivo_pickle(indice_invertido):
  nome_arquivo_base = f"{'../input/Raw Text/Rag Data/'}{ARQUIVO_INDICE_INVERTIDO}"

  with open(f'{nome_arquivo_base}_indice.pickle', 'wb') as f:
    pickle.dump(indice_invertido.indice, f)
  with open(f'{nome_arquivo_base}_tamanho_doc.pickle', 'wb') as f:
    pickle.dump(indice_invertido.tamanho_doc, f)

def recuperar_indice_em_arquivo_pickle():
  nome_arquivo_base = f"{'../input/Raw Text/Rag Data/'}{ARQUIVO_INDICE_INVERTIDO}"
  idx = IndiceInvertido()

  with open(f'{nome_arquivo_base}_indice.pickle', 'rb') as f:
    idx.indice = pickle.load(f)
  with open(f'{nome_arquivo_base}_tamanho_doc.pickle', 'rb') as f:
    idx.tamanho_doc = pickle.load(f)
  idx.n_docs = len(idx.tamanho_doc)

  return idx

In [67]:
%%time
iidx = IndiceInvertido()

if GERAR_INDICE_INVERTIDO:
  # Vamos usar o próprio índice como um id para o segmento. Vai facilitar a vida depois
  # art_seg['contents'] tem o título e 3 frases do artigo
  for id, art_seg in enumerate(artigos_segmentados):
    iidx.adiciona_doc(id, art_seg['contents'])

    if id % 10000 == 0:
      print(f'{id} segmentos indexados')

  salvar_indice_em_arquivo_pickle(iidx)
else:
  iidx = recuperar_indice_em_arquivo_pickle()

0 segmentos indexados
CPU times: total: 34.6 s
Wall time: 51.9 s


In [ ]:
for id, art_seg in enumerate(artigos_segmentados):
    iidx.adiciona_doc(id, art_seg['contents'])

    if id % 100 == 0:
      print(f'{id} segmentos indexados')

In [68]:
buscador = BM25(iidx, 0.82, 0.68, 1)
buscador = BM25(iidx)

In [69]:
print('Pergunta e resposta esperada: ')
print(perg_resposta_esperada[1])

resultado_bm25 = buscador.pesquisar(perg_resposta_esperada[1]['pergunta'])

print('-----------------------------------------------------------------')
print('10 primeiros conteúdos retornados pelo BM25\n')
for i in range(5):
  segmento_id = resultado_bm25[i][0]
  score = resultado_bm25[i][1]

  title = artigos_segmentados[segmento_id]['title']
  contents = artigos_segmentados[segmento_id]['contents']
  segment = artigos_segmentados[segmento_id]['segment']

  print(f'Segmento_id: {segmento_id}. Score: {score}')
  print(f"Title: {title}")
  print(f"Contents: {contents}")
  print('-----------------------------------------------------------------')

Pergunta e resposta esperada: 
{'pergunta': 'Como ocorrem as atividades de cooperação, pesquisa, ensino, extensão e prestação de serviços entre a Unicamp e terceiros, pessoas físicas ou jurídicas?', 'resposta': 'Nos termos do art. 1° da Deliberação CONSU-A-016/2022, a atuação da Universidade em atividades de cooperação, pesquisa, ensino, extensão e prestação de serviços junto a terceiros, pessoas físicas ou jurídicas, deve ocorrer mediante prévia celebração de convênios, contratos e instrumentos similares.'}
-----------------------------------------------------------------
10 primeiros conteúdos retornados pelo BM25

Segmento_id: 1400. Score: -3.803192377090454
Title: Constituição Federal
Contents: Constituição Federal. Parágrafo único. A lei garantirá tratamento especial à propriedade produtiva e fixará normas para o cumprimento dos requisitos relativos a sua função social.   Art. 186.
-----------------------------------------------------------------
Segmento_id: 1119. Score: -4.6424